In [1]:
import os

os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/worktalksbyai/datascienceproject.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="worktalksbyai"
os.environ["MLFLOW_TRACKING_PASSWORD"]="4d592b43bd74b433231cf8760e6eac201a31c0f1"

In [2]:
os.chdir("../")
%pwd

'/Users/aple/Documents/VSCode_GitHub/MLOps Bootcamp/Data Science Project/datascienceproject'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [4]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories, save_json

In [5]:
class ConfigurationManager:
    def __init__(self,
                config_filepath=CONFIG_FILE_PATH,
                params_filepath=PARAMS_FILE_PATH,
                schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/worktalksbyai/datascienceproject.mlflow"
        )
        return model_evaluation_config

In [ ]:
import os
import pandas as pd
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow==2.0
import mlflow.sklearn
import numpy as np
import joblib

In [19]:
class ModelEvaluation:
    def __init__(self,config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual,pred):
        #rmse = root_mean_squared_error(actual, pred)
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return mae, r2

    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            #Saving metrics as local
            scores = {"mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            #mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            # Model Registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [20]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-07-04 00:16:59,130: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-07-04 00:16:59,134: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-04 00:16:59,162: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-07-04 00:16:59,168: INFO: common: Created directory at: artifacts]
[2025-07-04 00:16:59,172: INFO: common: Created directory at: artifacts/model_evaluation]
[2025-07-04 00:16:59,608: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]


2025/07/04 00:17:02 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run fun-auk-957 at: https://dagshub.com/worktalksbyai/datascienceproject.mlflow/#/experiments/0/runs/1735c444f3b1478988837731d3baeb29
🧪 View experiment at: https://dagshub.com/worktalksbyai/datascienceproject.mlflow/#/experiments/0


RestException: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}